In [1]:
!pip install gspread
!pip install oauth2client

  Using cached oauth2client-4.1.3-py2.py3-none-any.whl (98 kB)
  Using cached httplib2-0.19.1-py3-none-any.whl (95 kB)


In [2]:
import pandas as pd
from datetime import datetime
import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [10]:
def auth():
    SP_CREDENTIAL_FILE = 'secret.json'
    SP_SCOPE = [
        'https://spreadsheets.google.com/feeds',
        'https://www.googleapis.com/auth/drive'
    ]

    SP_SHEET_KEY = '1gyUDuEZnmBi8kEzyUHCzMrEEj-ZlJAjDGpPOiM47FRQ'
    SP_SHEET = 'timesleet'

    credentials = ServiceAccountCredentials.from_json_keyfile_name(SP_CREDENTIAL_FILE, SP_SCOPE)
    gc = gspread.authorize(credentials)

    worksheet = gc.open_by_key(SP_SHEET_KEY).worksheet(SP_SHEET)
    return worksheet
  

In [11]:
worksheet = auth()

In [12]:
timestamp = datetime.now()

In [16]:
date = timestamp.strftime('%Y/%m/%d')
date

'2021/05/12'

In [17]:
punch_in = timestamp.strftime('%H:%M')
punch_in

'18:04'

In [19]:
df = pd.DataFrame(worksheet.get_all_records())
df

,日付,出勤時間,退勤時間
0,2021/05/12,10:00,18:00


In [20]:
df = df.append({'日付': date, '出勤時間': punch_in, '退勤時間': '00:00'}, ignore_index=True)
df

,日付,出勤時間,退勤時間
0,2021/05/12,10:00,18:00
1,2021/05/12,18:04,00:00


In [21]:
worksheet.update([df.columns.values.tolist()] + df.values.tolist())

{'spreadsheetId': '1gyUDuEZnmBi8kEzyUHCzMrEEj-ZlJAjDGpPOiM47FRQ',
 'updatedRange': 'timesleet!A1:C3',
 'updatedRows': 3,
 'updatedColumns': 3,
 'updatedCells': 9}

In [22]:
timestamp = datetime.now()

In [23]:
punch_out = timestamp.strftime('%H:%M')
punch_out

'18:15'

In [25]:
df.iloc[-1, 2] = punch_out

In [26]:
df

,日付,出勤時間,退勤時間
0,2021/05/12,10:00,18:00
1,2021/05/12,18:04,18:15


In [27]:
worksheet.update([df.columns.values.tolist()] + df.values.tolist())

{'spreadsheetId': '1gyUDuEZnmBi8kEzyUHCzMrEEj-ZlJAjDGpPOiM47FRQ',
 'updatedRange': 'timesleet!A1:C3',
 'updatedRows': 3,
 'updatedColumns': 3,
 'updatedCells': 9}

In [28]:
import pandas as pd
from datetime import datetime
import gspread
from oauth2client.service_account import ServiceAccountCredentials

def auth():
    SP_CREDENTIAL_FILE = 'secret.json'
    SP_SCOPE = [
        'https://spreadsheets.google.com/feeds',
        'https://www.googleapis.com/auth/drive'
    ]

    SP_SHEET_KEY = '1gyUDuEZnmBi8kEzyUHCzMrEEj-ZlJAjDGpPOiM47FRQ'
    SP_SHEET = 'timesleet'

    credentials = ServiceAccountCredentials.from_json_keyfile_name(SP_CREDENTIAL_FILE, SP_SCOPE)
    gc = gspread.authorize(credentials)

    worksheet = gc.open_by_key(SP_SHEET_KEY).worksheet(SP_SHEET)
    return worksheet

# 出勤
def punch_in():
    worksheet = auth()
    df = pd.DataFrame(worksheet.get_all_records())

    timestamp = datetime.now()
    date = timestamp.strftime('%Y/%m/%d')
    punch_in = timestamp.strftime('%H:%M')

    df = df.append({'日付': date, '出勤時間': punch_in, '退勤時間': '00:00'}, ignore_index=True)
    worksheet.update([df.columns.values.tolist()] + df.values.tolist())
    print('勤怠（出勤）登録完了しました')

# 退勤
def punch_out():
    worksheet = auth()
    df = pd.DataFrame(worksheet.get_all_records())

    timestamp = datetime.now()
    punch_out = timestamp.strftime('%H:%M')

    df.iloc[-1, 2] = punch_out
    worksheet.update([df.columns.values.tolist()] + df.values.tolist())
    print('勤怠（退勤）登録完了しました')

In [29]:
punch_in()

勤怠（出勤）登録完了しました


In [30]:
punch_out()

勤怠（退勤）登録完了しました
